In [ ]:
import numpy as np
import timeit

import matplotlib.pyplot as plt

import torch
import torch.optim as optim

import sys
sys.path.insert(0, './../')
import sdss_psf
import simulated_datasets_lib
import starnet_vae_lib

import objectives_lib

import time

import json

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

print('torch version: ', torch.__version__)

from copy import deepcopy

In [ ]:
# load PSF
psf_fit_file = '../../celeste_net/sdss_stage_dir/3900/6/269/psField-003900-6-0269.fit'
print('psf file: \n', psf_fit_file)

In [ ]:
np.random.seed(32)
_ = torch.manual_seed(54)

# Draw data

In [ ]:
# data parameters
with open('../data/default_star_parameters.json', 'r') as fp:
    data_params = json.load(fp)

data_params['min_stars'] = 0
data_params['max_stars'] = 4

print(data_params)


In [ ]:
max_stars = data_params['max_stars']

In [ ]:
batchsize = 2048

star_dataset = \
    simulated_datasets_lib.load_dataset_from_params(psf_fit_file,
                            data_params,
                            n_stars = batchsize,
                            use_fresh_data = False, 
                            add_noise = True)

num_unlabeled = star_dataset.fluxes.shape[0]
print('num unlabeled', num_unlabeled)

In [ ]:
# true parameters
test_loader = torch.utils.data.DataLoader(
                 dataset=star_dataset,
                 batch_size=batchsize,
                 shuffle=False)

for _, data in enumerate(test_loader):
    true_fluxes = data['fluxes']
    true_locs = data['locs']
    true_n_stars = data['n_stars']
    images = data['image']
    backgrounds = data['background']
    
    break

In [ ]:
true_is_on = simulated_datasets_lib.get_is_on_from_n_stars(true_n_stars, max_stars)

In [ ]:
images.shape

In [ ]:
# histogram of fluxes
plt.hist(true_fluxes.numpy().flatten()); 

In [ ]:
_, axarr = plt.subplots(2, 5, figsize=(18, 8))
for i in range(0, 10): 
    
    i1 = int(np.floor(i / 5))
    i2 = i % 5
        
    # image 
    axarr[i1, i2].matshow(images[i, 0, :, :])
    axarr[i1, i2].set_title('n_stars: {}\n'.format(true_n_stars[i]))
    
    # plot locations 
    locs_i = true_locs[i]
    n_stars_i = true_n_stars[i]
    locs_y = (locs_i[0:int(n_stars_i), 0]) * (images.shape[-1] - 1) 
    locs_x = (locs_i[0:int(n_stars_i), 1]) * (images.shape[-1] - 1)
    
    axarr[i1, i2].scatter(x = locs_x, y = locs_y, color = 'b')


# Load VAE

In [ ]:
star_encoder = starnet_vae_lib.StarEncoder(data_params['slen'], 
                                           n_bands = 1, 
                                          max_detections = max_stars)

In [ ]:
star_encoder.load_state_dict(torch.load('../fits/starnet_invKL_encoder_four_stars', 
                               map_location=lambda storage, loc: storage))
star_encoder.eval(); 

In [ ]:
objectives_lib.eval_star_encoder_loss(star_encoder, test_loader, train = False)

# elbo_objective_lib.eval_star_encoder_kl_loss(star_encoder, test_loader, train = False)

In [ ]:
loss, locs_loss, fluxes_loss, perm = \
    objectives_lib.get_encoder_loss(star_encoder, images, backgrounds, true_locs,
                        true_fluxes, true_n_stars)
    
print(loss)

In [ ]:
plt.hist(locs_loss.detach().numpy().flatten());

In [ ]:
plt.hist(fluxes_loss.detach().numpy().flatten()); 

# check parameters

In [ ]:
# permute true parameters 
def permute_params(true_locs, true_fluxes, perm): 
    batchsize = true_locs.shape[0]
    max_stars = true_locs.shape[1]

    locs_perm = torch.zeros((batchsize, max_stars, 2))
    fluxes_perm = torch.zeros((batchsize, max_stars))
    seq_tensor = torch.LongTensor([i for i in range(batchsize)])

    for i in range(max_stars):
        locs_perm[:, i, :] = true_locs[seq_tensor, perm[:, i], :]
        fluxes_perm[:, i] = true_fluxes[seq_tensor, perm[:, i]]
        
    return locs_perm, fluxes_perm

In [ ]:
locs_perm, fluxes_perm = permute_params(true_locs, true_fluxes, perm)

In [ ]:
# get variational parameters
logit_loc_mean, logit_loc_log_var, \
        log_flux_mean, log_flux_log_var = star_encoder(images, backgrounds, true_n_stars)

In [ ]:
plt.hist(log_flux_log_var.flatten().detach().numpy(), bins = 100);

In [ ]:
plt.hist(logit_loc_log_var.flatten().detach().numpy(), bins = 100);

In [ ]:
map_locs = torch.sigmoid(logit_loc_mean)
map_fluxes = torch.exp(log_flux_mean)

In [ ]:
# error in locs
for i in range(max_stars): 
    
    is_on_i = true_is_on[:, i]
    
    
    plt.plot(map_locs[is_on_i == 1, i, 0].detach().numpy(), 
                locs_perm[is_on_i == 1, i, 0].detach().numpy(), '+', color = 'blue', alpha = 0.05)
    plt.plot(map_locs[is_on_i == 1, i, 0].detach().numpy(), 
                 map_locs[is_on_i == 1, i, 0].detach().numpy(), color = 'red')
    plt.xlabel('Estimated x coordinate', fontsize = 16)
    plt.ylabel('True x coordinate', fontsize = 16)


In [ ]:
# error in locs
for i in range(max_stars): 
    
    is_on_i = true_is_on[:, i]
    
    
    plt.plot(map_locs[is_on_i == 1, i, 1].detach().numpy(), 
                locs_perm[is_on_i == 1, i, 1].detach().numpy(), '+', color = 'blue', alpha = 0.05)
    plt.plot(map_locs[is_on_i == 1, i, 1].detach().numpy(), 
                 map_locs[is_on_i == 1, i, 1].detach().numpy(), color = 'red')
    plt.xlabel('Estimated y coordinate', fontsize = 16)
    plt.ylabel('True y coordinate', fontsize = 16)



In [ ]:
for i in range(max_stars): 
   
    is_on_i = true_is_on[:, i]
#     plt.figure()
    
    plt.plot(np.log(map_fluxes[is_on_i == 1, i].detach().numpy()), 
             np.log(fluxes_perm[is_on_i == 1, i].detach().numpy()), '+', color = 'blue')
    
    plt.plot(np.log(map_fluxes[is_on_i == 1, i].detach().numpy()), 
             np.log(map_fluxes[is_on_i == 1, i].detach().numpy()), color = 'red')
    
    plt.xlabel('Estimated flux', fontsize = 16)
    plt.ylabel('True flux', fontsize = 16)
    



In [ ]:
# Check reconstructions 

In [ ]:
recon_images = star_dataset.draw_image_from_params(map_locs, map_fluxes, true_n_stars, add_noise = False)

In [ ]:
for i in range(0, 20): 
    _, axarr = plt.subplots(1, 3, figsize=(12, 4))

    # observed image 
    axarr[0].matshow(images[i, 0, :, :])
    axarr[0].set_title('Observed image \n locs loss: {:.04f}'.format(locs_loss[i]))
    
    # plot locations 
    locs_i = true_locs[i]
    n_stars_i = true_n_stars[i]
    locs_y = (locs_i[0:int(n_stars_i), 0]) * (images.shape[-1] - 1) 
    locs_x = (locs_i[0:int(n_stars_i), 1]) * (images.shape[-1] - 1)
    
    axarr[0].scatter(x = locs_x, y = locs_y, c = 'b')
    axarr[0].get_xaxis().set_visible(False)
    axarr[0].get_yaxis().set_visible(False)
    
    # plot estimated locations 
    map_locs_i = map_locs[i].detach()
    n_stars_i = true_n_stars[i]
    est_locs_y = (map_locs_i[0:int(n_stars_i), 0]) * (images.shape[-1] - 1) 
    est_locs_x = (map_locs_i[0:int(n_stars_i), 1]) * (images.shape[-1] - 1)
    
    axarr[0].scatter(x = est_locs_x, y = est_locs_y, c = 'r', marker = 'x')
    
    # plot posterior samples
    axarr[1].matshow(images[i, 0, :, :].detach())
    axarr[1].set_title('Observed image \n locs loss: {:.04f}'.format(locs_loss[i]))
    axarr[1].get_xaxis().set_visible(False)
    axarr[1].get_yaxis().set_visible(False)
    

    for k in range(int(n_stars_i)): 
        samples = torch.sigmoid(torch.sqrt(torch.exp(logit_loc_log_var[i, k, :])) * \
                      torch.randn((1000, 2)) + logit_loc_mean[i, k, :]).detach()
        
        axarr[1].scatter(x = samples[:, 1] * (images.shape[-1] - 1) , 
                         y = samples[:, 0] * (images.shape[-1] - 1) , 
                         c = 'r', marker = 'x', alpha = 0.05)
    axarr[1].scatter(x = locs_x, y = locs_y, c = 'b')
    
    # plot residuals
    axarr[2].matshow(images[i, 0, :, :] - recon_images[i, 0, :, :].detach())
    axarr[2].get_xaxis().set_visible(False)
    axarr[2].get_yaxis().set_visible(False)
    axarr[2].set_title('residuals')
    


In [ ]:
# results for a specific n_stars
for i in range(0, 80): 
    
    if true_n_stars[i] != 4: 
        continue 
        
    _, axarr = plt.subplots(1, 3, figsize=(12, 4))

    # observed image 
    axarr[0].matshow(images[i, 0, :, :])
    axarr[0].set_title('Observed image \n locs loss: {:.04f}'.format(locs_loss[i]))
    
    # plot locations 
    locs_i = true_locs[i]
    n_stars_i = true_n_stars[i]
    locs_y = (locs_i[0:int(n_stars_i), 0]) * (images.shape[-1] - 1) 
    locs_x = (locs_i[0:int(n_stars_i), 1]) * (images.shape[-1] - 1)
    
    axarr[0].scatter(x = locs_x, y = locs_y, c = 'b')
    axarr[0].get_xaxis().set_visible(False)
    axarr[0].get_yaxis().set_visible(False)
    
    # plot estimated locations 
    map_locs_i = map_locs[i].detach()
    n_stars_i = true_n_stars[i]
    est_locs_y = (map_locs_i[0:int(n_stars_i), 0]) * (images.shape[-1] - 1) 
    est_locs_x = (map_locs_i[0:int(n_stars_i), 1]) * (images.shape[-1] - 1)
    
    axarr[0].scatter(x = est_locs_x, y = est_locs_y, c = 'r', marker = 'x')
    
    # plot posterior samples
    axarr[1].matshow(images[i, 0, :, :].detach())
    axarr[1].set_title('Observed image \n locs loss: {:.04f}'.format(locs_loss[i]))
    axarr[1].get_xaxis().set_visible(False)
    axarr[1].get_yaxis().set_visible(False)
    

    for k in range(int(n_stars_i)): 
        samples = torch.sigmoid(torch.sqrt(torch.exp(logit_loc_log_var[i, k, :])) * \
                      torch.randn((1000, 2)) + logit_loc_mean[i, k, :]).detach()
        
        axarr[1].scatter(x = samples[:, 1] * (images.shape[-1] - 1) , 
                         y = samples[:, 0] * (images.shape[-1] - 1) , 
                         c = 'r', marker = 'x', alpha = 0.05)
    axarr[1].scatter(x = locs_x, y = locs_y, c = 'b')
    
    # plot residuals
    axarr[2].matshow(images[i, 0, :, :] - recon_images[i, 0, :, :].detach())
    axarr[2].get_xaxis().set_visible(False)
    axarr[2].get_yaxis().set_visible(False)
    axarr[2].set_title('residuals')
    
